In [1]:
# example of training an conditional gan on the fashion mnist dataset
#RICORDA DIMENSIONE LATENT SPACE
import tensorflow as tf
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten, MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, ReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageOps
import os
import time
import matplotlib.pyplot as plt
import cv2

Using TensorFlow backend.


In [2]:
DATA_PATH = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpacganfirstattempt/code/Users/10539060'
image_size = 256

# inserire immagini come np.array
training_images_path = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpacganfirstattempt/code/Users/10539060/images_256_256_MD.npy'

# inserire classi come np.array Users/10539060/images_256_256_MD.npy
training_labels_path = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpacganfirstattempt/code/Users/10539060/labels_256_256_MD.npy'

cwd = os.getcwd()
print(cwd)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpacganfirstattempt/code/Users/10539060


In [3]:
# define the standalone discriminator model
def define_discriminator(in_shape=(256,256,3), n_classes=4):
        # label input
        in_label = Input(shape=(1,))
        # embedding for categorical input
        li = Embedding(n_classes, 35)(in_label)
        # scale up to image dimensions with linear activation
        n_nodes = in_shape[0] * in_shape[1]
        li = Dense(n_nodes)(li)
        # reshape to additional channel
        li = Reshape((in_shape[0], in_shape[1], 1))(li)
        # image input
        in_image = Input(shape=in_shape)
        #in_image = Reshape(input_shape=(in_shape), target_shape=(256,256,3))(in_image)
        # concat label as a channel
        merge = Concatenate()([in_image, li])
        fe = Conv2D(256, (3,3), strides=(2,2), padding='same')(merge)
        fe = LeakyReLU(alpha=0.2)(fe) #prima non c'era
        # downsample
        fe = Conv2D(512, (3,3), strides=(2,2), padding='same')(fe)
        fe = LeakyReLU(alpha=0.2)(fe)
        # flatten feature maps
        fe = Flatten()(fe)
        # dropout
        fe = Dropout(0.4)(fe)
        # output
        out_layer = Dense(1, activation='sigmoid')(fe)
        # define model
        model = Model([in_image, in_label], out_layer)
        # compile model
        opt = Adam(lr=0.0002, beta_1=0.5) 
        model.summary()
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model

# define the standalone generator model: proviamo a fare conv+up invece che convtranspose
def define_generator(latent_dim, size, n_classes=4):
    # label input
        in_label = Input(shape=(1,))
        # embedding for categorical input
        li = Embedding(n_classes, 35)(in_label)
        # linear multiplication
        n_nodes = int(size/4) * int(size/4)
        li = Dense(n_nodes)(li)
        # reshape to additional channel
        li = Reshape((int(size/4), int(size/4), 1))(li)
        # image generator input
        in_lat = Input(shape=(latent_dim,))
        # foundation for 7x7 image
        n_nodes = 128 * int(size/4) * int(size/4)
        gen = Dense(n_nodes)(in_lat)
        gen = LeakyReLU(alpha=0.2)(gen)
        gen = Reshape((int(size/4), int(size/4), 128))(gen)
        # merge image gen and label input
        merge = Concatenate()([gen, li])
        # upsample to 14x14 provo a mettere più filtri
        gen = Conv2D(512, (3,3), padding='same')(merge) # strides=(2,2), 
        gen = LeakyReLU(alpha=0.3)(gen)
        #gen = MaxPooling2D()(gen) #era commentato
        gen = UpSampling2D()(gen)
        # upsample to 28x28
        gen = Conv2D(256, (3,3), padding='same')(gen) # strides=(2,2), 
        gen = LeakyReLU(alpha=0.3)(gen)
        gen = UpSampling2D()(gen)
        # output
        out_layer = Conv2D(3, (7,7), activation='tanh', padding='same')(gen)
        # define model
        model = Model([in_lat, in_label], out_layer)
        model.summary()
        return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label= g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [4]:
def load_real_samples():
  trainX = np.load(training_images_path, allow_pickle = True)
  trainy = np.load(training_labels_path, allow_pickle = True)
  #Adding noise to input data
  X = trainX.astype('float32')
  X = (X - 127.5) / 127.5 + np.random.normal(0, 0.01, (trainX.shape[0],trainX.shape[1],trainX.shape[2],trainX.shape[3]))
  #From str labels to integers
  for i in range(len(trainy)):
    if trainy[i] == 'Happy':
      trainy[i] = np.int(0)
    if trainy[i] == 'Angry':
      trainy[i] = np.int(1)
    if trainy[i] == 'Sad':
      trainy[i] = np.int(2)
    if trainy[i] == 'Relaxed':
      trainy[i] = np.int(3)
  trainy = trainy.astype('float32')
  return[X, trainy]

In [5]:
# # select real samples
def generate_real_samples(dataset, n_samples):
  # split into images and labels
  images, labels = dataset
  # choose random instances
  ix = randint(0, images.shape[0], n_samples)
  # select images and labels
  X, labels = images[ix], labels[ix]
  # generate class labels
  y = ones((n_samples, 1))
  # labels switching
  for i in range(int(n_samples/3)):
    y[3*i] = 0 
  return [X, labels], y

In [6]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=4):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

In [7]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
  # generate points in latent space
  z_input, labels_input = generate_latent_points(latent_dim, n_samples)
  # predict outputs
  images = generator.predict([z_input, labels_input])
  # create class labels
  y = zeros((n_samples, 1))
  #labels switching
  for i in range(int(n_samples/3)):
    y[3*i] = 1
  return [images, labels_input], y

In [8]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=25, n_batch=16): #batch era 128
  bat_per_epo = int(dataset[0].shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
      # get randomly selected 'real' samples
      [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
      # update discriminator model weights
      d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
      # generate 'fake' examples
      [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      # update discriminator model weights
      d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
      # prepare points in latent space as input for the generator
      [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
      # summarize loss on this batch
      print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
        (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))

In [9]:
# size of the latent space
latent_dim = 16
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim, image_size)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 35)        140         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1, 65536)     2359296     embedding_1[0][0]                
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
____________________________________________________________________________________________

In [10]:
# train and save model
train(g_model, d_model, gan_model, dataset, latent_dim)
g_model.save(os.path.join(DATA_PATH, f'generator_{image_size}_100_new.h5'))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/

>1, 1/465, d1=0.694, d2=0.700 g=0.700
>1, 2/465, d1=1.458, d2=0.715 g=0.699
>1, 3/465, d1=0.648, d2=0.736 g=0.673
>1, 4/465, d1=0.717, d2=0.647 g=1.041
>1, 5/465, d1=1.295, d2=0.696 g=0.650
>1, 6/465, d1=0.783, d2=0.720 g=0.713
>1, 7/465, d1=0.478, d2=0.655 g=0.942
>1, 8/465, d1=1.084, d2=0.673 g=0.746
>1, 9/465, d1=0.503, d2=0.685 g=0.731
>1, 10/465, d1=0.383, d2=0.717 g=0.785
>1, 11/465, d1=1.474, d2=0.627 g=1.156
>1, 12/465, d1=0.552, d2=0.591 g=1.048
>1, 13/465, d1=1.079, d2=0.669 g=0.780
>1, 14/465, d1=0.767, d2=0.682 g=0.752
>1, 15/465, d1=0.974, d2=0.705 g=0.690
>1, 16/465, d1=0.680, d2=0.730 g=0.719
>1, 17/465, d1=0.546, d2=0.697 g=0.952
>1, 18/465, d1=0.647, d2=0.641 g=0.968
>1, 19/465, d1=0.901, d2=0.651 g=1.003
>1, 20/465, d1=0.702, d2=0.695 g=0.779
>1, 21/465, d1=0.619, d2=0.666 g=0.875
>1, 22/465, d1=0.674, d2=0.609 g=1.112
>1, 23/465, d1=0.640, d2=0.618 g=0.953
>1, 24/465, d1=0.489, d2=0.631 g=0.937
>1, 25/465, d1=0.803, d2=0.687 g=0.866
>1, 26/465, d1=0.308, d2=0.805 g=0